In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [4]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = int(dataset_size * 0.9)
n_steps = 100
window_length = n_steps + 1

In [5]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda x_batch, y_batch: (tf.one_hot(x_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [6]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, 
                     batch_input_shape=[1, None, max_id], 
                     dropout=0.2, stateful=True), 
    keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, stateful=True), 
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation='softmax'))
])

In [7]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [8]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

history = model.fit(dataset, epochs=5, callbacks=[ResetStatesCallback()])

Epoch 1/5
10038/10038 [==============================] - 79s 8ms/step - loss: 2.0886
Epoch 2/5
10038/10038 [==============================] - 73s 7ms/step - loss: 1.8959
Epoch 3/5
10038/10038 [==============================] - 73s 7ms/step - loss: 1.8458
Epoch 4/5
10038/10038 [==============================] - 78s 8ms/step - loss: 1.8184
Epoch 5/5
10038/10038 [==============================] - 76s 8ms/step - loss: 1.7992


In [9]:
def preprocess(texts):
    x = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(x, max_id)

def next_char(text, temperature=1):
    x_new = preprocess([text])
    y_proba = model(x_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [10]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation='softmax'))
])

In [11]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [12]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [13]:
complete_text('t', temperature=0.2)

'the world her for my father,\nand she shall be the s'

In [14]:
complete_text('t', temperature=1)

'to clare it would him\nit is been of\nyour dighters y'

In [15]:
complete_text('t', temperature=2)

"tis?\nby kope m! her yeajy'd; the mea. cspuls fac yo"

In [16]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)

datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)

dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda x_batch, y_batch: (tf.one_hot(x_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [17]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, 
                     batch_input_shape=[batch_size, None, max_id], 
                     dropout=0.2, stateful=True), 
    keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, stateful=True), 
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 5s 11ms/step - loss: 2.6124
Epoch 2/50
313/313 [==============================] - 3s 11ms/step - loss: 2.2407
Epoch 3/50
313/313 [==============================] - 3s 10ms/step - loss: 2.1119
Epoch 4/50
313/313 [==============================] - 3s 10ms/step - loss: 2.0353
Epoch 5/50
313/313 [==============================] - 3s 11ms/step - loss: 1.9857
Epoch 6/50
313/313 [==============================] - 3s 10ms/step - loss: 1.9471
Epoch 7/50
313/313 [==============================] - 3s 10ms/step - loss: 1.9186
Epoch 8/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8967
Epoch 9/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8780
Epoch 10/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8653
Epoch 11/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8513
Epoch 12/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8390
Epoch 13/50
3

In [18]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation='softmax'))
])

stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [19]:
complete_text('t', temperature=0.2)

'there we will be a seed\nto his son to the fire of a'

In [20]:
complete_text('t', temperature=1)

'thore,\nby pacred but bark you land true from him.\ns'

In [21]:
complete_text('t', temperature=2)

'tceoh\nekmlesiceor by miscrmns blyst ben?\nmat hust?e'